In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dependências e utilidades

In [2]:
import numpy as np
import glob
import cv2

from PIL import Image

# CNN
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

# Plot
from matplotlib import pyplot as plt
%matplotlib inline  

# ZIP
from zipfile import ZipFile 

np.random.seed(123)  # for reproducibility
captcha_size = 6

# Importando o dataset
O dataset fornecido está no arquivo `dados.zip`. O primeiro passo é descompactar o dataset para a pasta onde esta o notebook. Utilizando a biblioteca `glob` obtemos a lista de todos os arquivos de cada dataset (teste, validação, treinamento e labels).

In [3]:
dataset_file_name = "/content/drive/My Drive/TP3-ICV/dados.zip"
  
# Extrair o dataset do arquivo compactado 
ZipFile(dataset_file_name, 'r').extractall() 

# Filelist dos dados
test_filelist   = sorted(glob.glob('/content/dados/CAPTCHA-10k/teste/*.jpg'))
valid_filelist  = sorted(glob.glob('/content/dados/CAPTCHA-10k/validacao/*.jpg'))
train_filelist  = sorted(glob.glob('/content/dados/CAPTCHA-10k/treinamento/*.jpg'))
labels_filelist = sorted(glob.glob('/content/dados/CAPTCHA-10k/labels10k/*.txt'))

## Importando as imagens
Em seguida, abrimos todas as imagens e as armazenamos em arrays separados.

In [4]:
# Importar imagens
test  = np.array([np.array(Image.open(file).getchannel('R')) for file in test_filelist])
valid = np.array([np.array(Image.open(file).getchannel('R')) for file in valid_filelist])
train = np.array([np.array(Image.open(file).getchannel('R')) for file in train_filelist])

## Importando os labels
Importamos os labels e armazenamos todos em uma única lista. Essa lista é dividida em 3, cada uma contendo os labels de um dataset (teste, validação e treinamento).

In [5]:
# Importar labels
all_labels = [open(file, "r").read(captcha_size) for file in labels_filelist]
test_lbs  = all_labels[9000:10000]
valid_lbs = all_labels[8000:9000]
train_lbs = all_labels[:8000]

In [ ]:
plt.title(test_lbs[0])
plt.imshow(test[0], cmap='gray')
plt.show()

# Tratamento dos dados


## Remoção dos dados mal formados
O dataset fornecido possui alguns labels que contem o caractere `?`, que não corresponde às imagens dos CAPTCHAS. Por isso, precisamos remover todos os pares de dados que contém estes dados mal formados.

In [7]:
# Remover captchas com '?'
def removeInterrogation(images, labels):
    temp_lb = []
    temp_im = []
    for i in range(len(labels)):
        s = list(labels[i])
        if ('?' not in s) and len(s) == captcha_size:
            temp_im.append(images[i])
            temp_lb.append(labels[i])
    return np.array(temp_im), temp_lb

test,  test_lbs  = removeInterrogation(test,  test_lbs)
train, train_lbs = removeInterrogation(train, train_lbs)
valid, valid_lbs = removeInterrogation(valid, valid_lbs)

## Recortando os CAPTCHAS
Nossa rede neural será treinada para reconhecer caracteres individuais dos CAPTCHAS. Por isso, devemos cortar cada imagem em 6 imagens, cada uma contendo um caractere do captcha. Nossa abrodagem foi bem simples:
1. Removemos 12 primeiras colunas da imagem, que geralmente contem apenas espaço em branco.
2. Dividimos o restante em 6 partes iguais, com dimensões 28x50

In [8]:
# Dimensões das novas imagens
height, width = 50, 28

def cropCaptchas(arr):
    arr_cpd = []
    for im in arr:
        #captcha = enhanceCaptcha(im)
        captcha = im[:,12:]
        for i in range(captcha_size):
            arr_cpd.append(captcha[:,width*i:width*i+width])
    return np.array(arr_cpd)

test_cpd  = cropCaptchas(test)
train_cpd = cropCaptchas(train)
valid_cpd = cropCaptchas(valid)

## Recortando os labels
Agora que recortamos os CAPTCHAS em caracteres individuais, vamos fazer o mesmo com os labels.

In [9]:
def cropLabels(arr):
    arr_cpd = []
    for s in arr:
        s = list(s)
        for i in range(captcha_size):
            arr_cpd.append(s[i])
    return arr_cpd

test_lbs_cpd  = cropLabels(test_lbs)
train_lbs_cpd = cropLabels(train_lbs)
valid_lbs_cpd = cropLabels(valid_lbs)

## ~Tratamento de imagem~
Na tentativa de melhorar a acurácia do modelo, nós tentamos utilizar técnicas de processamento digital de imagens para realçar as imagens. **Porém, não obtivemos sucesso, e descartamos a tentativa**.

In [10]:
def enhanceCaptcha(img):
    img = img.astype('float32')
    #plt.imshow(img, cmap='gray')
    #plt.show()
    blur = cv2.GaussianBlur(img,(3,3),0)
    #plt.imshow(blur, cmap='gray')
    #plt.show()
    ret, thresh = cv2.threshold(img,50,255,cv2.THRESH_BINARY)
    #plt.imshow(thresh, cmap='gray')
    #plt.show()
    return thresh

## Preprocessamento das imagens de entrada para o Keras
Agora, preparamos as imagens para serem fornecidas à rede. Para isso, precisamos garantir que os arrays tem o `shape` correto, e que todos os valores das imagens estão no intervalo `[0,1]`.

In [ ]:
# Reshape dos arrays
test_cpd  = test_cpd.reshape(test_cpd.shape[0],   height, width, 1)
train_cpd = train_cpd.reshape(train_cpd.shape[0], height, width, 1)
valid_cpd = valid_cpd.reshape(valid_cpd.shape[0], height, width, 1)

print("Teste:\t\t{0}\nTreinamento:\t{1}\nValidação:\t{2}".format(test_cpd.shape, train_cpd.shape, valid_cpd.shape))

In [12]:
# Normalização dos valores dos pixels para [0,1]
test_cpd   = test_cpd.astype('float32') 
test_cpd  /= 255
train_cpd  = train_cpd.astype('float32')
train_cpd /= 255
valid_cpd  = valid_cpd.astype('float32') 
valid_cpd /= 255

## Preprocessamento dos labels das classes para o Kera
A nossa rede recebe os labels na forma de um array binário que indica a classe. Portanto, devemos transormar cada caractere em um array binário que contem um `1` apenas na posição que indica sua classe em relação à lista `captcha_chars`

In [13]:
captcha_chars = list('0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
num_chars = len(captcha_chars)
# Categorizar os labels
def toCategorical(arr):
    cat = []
    for c in arr:
        vec = np.zeros((num_chars), dtype='float32')
        vec[captcha_chars.index(c)] = 1.
        cat.append(vec)
    return np.array(cat)

In [ ]:
test_lbs_cat  = toCategorical(test_lbs_cpd)
valid_lbs_cat = toCategorical(valid_lbs_cpd)
train_lbs_cat = toCategorical(train_lbs_cpd)

print("ID da classe: ", test_lbs_cpd[2])
print("Output da rede: ", test_lbs_cat[2])

# Redes neurais
Depois de alguns testes, encontramos dois modelos com acurácia próxima, mas que diferem no número de parâmetros. 


## Modelo 1
- C32(5)-MP-C48(4)-MP-C64(4)-MP-F-De128-Dr30-De36
- Numero de parametros: 308,820

### Arquitetura do modelo 1

In [34]:
# Sequential layer
model1 = Sequential()

In [35]:
model1.add(Conv2D(32, (5,5), strides=(1,1), activation='relu', padding='same', input_shape=(height,width,1)))
model1.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model1.add(Conv2D(48, (4,4), strides=(1,1), activation='relu', padding='same'))
model1.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model1.add(Conv2D(64, (4,4), strides=(1,1), activation='relu', padding='same'))
model1.add(MaxPooling2D(pool_size=(2,2), padding='same'))

In [36]:
# fully connected layer
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.3))
model1.add(Dense(36, activation='softmax'))

In [ ]:
model1.summary()

In [38]:
model1.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

### Treinamento do modelo 1

In [ ]:
model1.fit(train_cpd, train_lbs_cat, batch_size=128, epochs=30, verbose=1)

### Validação do modelo 1

In [ ]:
score1 = model1.evaluate(valid_cpd, valid_lbs_cat, verbose=0)
print("Modelo 1 score = {0}".format(score1))

## Modelo 2
- C64(5)-MP-C96(4)-MP-C128(4)-MP-F-De128-Dr30-De36
- Número de parâmetros: 760,324

### Arquitetura do modelo 2

In [22]:
# Sequential layer
model2 = Sequential()

In [23]:
model2.add(Conv2D(64, (5,5), strides=(1,1), activation='relu', padding='same', input_shape=(height,width,1)))
model2.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model2.add(Conv2D(96, (4,4), strides=(1,1), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model2.add(Conv2D(128, (4,4), strides=(1,1), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2,2), padding='same'))

In [24]:
# fully connected layer
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.3))
model2.add(Dense(36, activation='softmax'))

In [ ]:
model2.summary()

In [26]:
model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Treinamento do Modelo 2


In [ ]:
model2.fit(train_cpd, train_lbs_cat, batch_size=128, epochs=30, verbose=1)

### Validação do modelo 2

In [ ]:
score2 = model2.evaluate(valid_cpd, valid_lbs_cat, verbose=0)
print("Modelo 2 score = {0}".format(score2))

# Testes 
Com os modelos treinados, vamos agora aos testes.

In [41]:
# transformar as np array em letras
def fromCategorical(arr):
    arr = list(arr)
    i = arr.index(max(arr))
    return captcha_chars[i]

def runTest(model, id):
    predictions = model.predict(x=test_cpd, verbose=0)

    # Contar o numero de caracteres acertados em cada CAPTCHA
    acertos = []
    for i in range(len(test)):
        acerto = 0
        resposta = ''
        for j in range(captcha_size):
            c = fromCategorical(predictions[j+i*captcha_size])
            if c == test_lbs_cpd[j+i*captcha_size]:
                acerto += 1
            resposta += c
        acertos.append(acerto)
    '''
    plt.title(resposta +' '+ str(acerto))
    plt.imshow(test[i],cmap = "gray")
    plt.show()
    '''

    # Contar numero minimo de acertos
    count = []
    for i in range(captcha_size+1):
        count.append(acertos.count(i))

    # Calcular taxa de reconhecimento
    rate = []
    for i in range(captcha_size+1):
        rate.append(float(sum(count[i:])))
    rate = np.array(rate)
    rate/= rate[0]

    # Plotar gráfico
    plt.title("Resultado {0}".format(id))
    plt.ylim(0., 1.2)
    plt.xlabel('Número mínimo de caracteres reconhecidos por captcha')
    plt.ylabel('Taxa de Reconhecimento')
    plt.plot(list(range(captcha_size+1)), rate)

    for x,y in zip(list(range(captcha_size+1)),rate):
        label = "{:.3f}".format(y)
        plt.annotate(label, (x,y), textcoords="offset points", 
                     xytext=(0,6), ha='center') 
    plt.show()

In [ ]:
runTest(model1, 'Modelo 1')
runTest(model2, 'Modelo 2')